In [1]:
import sys
sys.path.append("/anaconda/envs/azureml_py38/lib/python3.8/site-packages")

In [2]:
import psycopg2
import pandas as pd
import numpy as np

In [3]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [4]:
conn, cur = get_db_cursor()


In [6]:
v1_samples = pd.read_csv('./outputFiles/spectra.csv',usecols=[0])

In [7]:
v1_samples

,Unnamed: 0
0,AGD207-408SA0706
1,AGD207-431SA0001
2,AGD207-431SA0002
3,AGD207-431SA0003
4,AGD207-431SA0004
...,...
87362,CW017SA7164
87363,CW017SA7167
87364,CW017SA7168
87365,CW104SA0052


In [8]:
v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)

/tmp/ipykernel_4146/2923516565.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)


In [9]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
185415,CA053SA1584
185416,CM754SA0017
185417,CA053SA1597
185418,CE445SA0001


In [10]:
v2_samples = v2_samples.loc[~v2_samples['sample_code'].isin(v1_samples['Unnamed: 0'])]

In [11]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
185415,CA053SA1584
185416,CM754SA0017
185417,CA053SA1597
185418,CE445SA0001


In [12]:
sample_codes = v2_samples['sample_code'].values

In [13]:
if(len(sample_codes) < 5000):
    count  = len(sample_codes)
    step=count
elif(len(sample_codes) < 70000):
    count = len(sample_codes)
    step=5000
else:
    count = 70000
    step=5000
start = 0


In [14]:
count

70000

In [15]:
start

0

In [16]:
step

5000

In [17]:
spectra = pd.DataFrame()

In [18]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [19]:
for i in np.arange(start, count, step):
        
    print("Fetching spectra from {}".format(start))
    samples = [i for i in sample_codes][start:start+step]
    print(len(samples))
    query = f"SELECT spectraldata.metadata_id, averaged_spectra, mandatorymetadata.sample_code  FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE is_finalized=True AND passed=True AND is_active=True AND averaged=True AND sample_code IN {str(samples).replace('[','(').replace(']',')')}"

    _ = pd.read_sql(query, con=conn)
    print("This is the spectra",_)
    spectra = pd.concat([spectra, _], axis=0)
    start = start + step
    if (count-step) > 5000:
        step=5000
    else:
        step = count-step

conn.close()
spectra = spectra[['sample_code', 'averaged_spectra']]
spectra = spectra.set_index('sample_code')
spectra = convertSpectra(spectra)


spectra.to_csv("./outputFiles/spectraldata.csv")



Fetching spectra from 0
5000
This is the spectra       metadata_id                                   averaged_spectra  \
0            3394  {'522': [2.152443884558668, 2.1388348361128093...   
1              17  {'522': [2.3243741634120374, 2.307540485180485...   
2              19  {'522': [2.2630152586026915, 2.290110779246487...   
3              47  {'522': [2.3843932844687363, 2.491238662094451...   
4            3311  {'522': [2.241984739726626, 2.1906742432163697...   
...           ...                                                ...   
4124       164706  {'522': [2.342822785838603, 2.3137618983017196...   
4125       181381  {'522': [2.273500591540199, 2.289040267833519,...   
4126       181387  {'522': [2.584770864487254, 2.487291039760919,...   
4127       181389  {'522': [2.393617302652631, 2.4448304558125797...   
4128       181390  {'522': [2.351067529303632, 2.365038585784037,...   

         sample_code  
0        CK013SA1874  
1        CS006SA0042  
2     FA419-51SA0

/tmp/ipykernel_4146/3563983685.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_4146/3563983685.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_4146/3563983685.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_4146/3563983685.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 o